# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [4]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [5]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings


generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [6]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [7]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [8]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 69)

We can save and load our knowledge graphs as follows.

In [9]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 69)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [10]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [11]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.2),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.4),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.4),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [12]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Wht has Anthropic contributed to the field of ...,[Code may be the best application The ethics o...,Anthropic has produced better-than-GPT-3 class...,single_hop_specifc_query_synthesizer
1,how python help with LLMs?,[Based Development As a computer scientist and...,Writing code is one of the things LLMs are mos...,single_hop_specifc_query_synthesizer
2,How have advancements in Large Language Models...,[<1-hop>\n\nCode may be the best application T...,Advancements in Large Language Models (LLMs) h...,multi_hop_abstract_query_synthesizer
3,How does the black box nature of Large Languag...,[<1-hop>\n\nCode may be the best application T...,The black box nature of Large Language Models ...,multi_hop_abstract_query_synthesizer
4,How have advancements in Large Language Models...,[<1-hop>\n\nCode may be the best application T...,Advancements in Large Language Models (LLMs) h...,multi_hop_abstract_query_synthesizer
5,What are the environmental impacts and trainin...,[<1-hop>\n\nCode may be the best application T...,The development of Large Language Models (LLMs...,multi_hop_abstract_query_synthesizer
6,How do the advancements in GPT-4 and GPT-4o re...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,The advancements in GPT-4 and GPT-4o highlight...,multi_hop_specific_query_synthesizer
7,What advancements in Claude 3 and Claude 3.5 S...,[<1-hop>\n\nreasoning patterns. Another common...,"Claude 3, launched by Anthropic in March, and ...",multi_hop_specific_query_synthesizer
8,What advancements in the development and appli...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2024, significant advancements in the devel...",multi_hop_specific_query_synthesizer
9,What were the major advancements and challenge...,[<1-hop>\n\neasy to follow. The rest of the do...,"In 2023, Large Language Models (LLMs) experien...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [8]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [14]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,what we learn about LLMs in 2024?,[Code may be the best application The ethics o...,"In 2024, we learned that Large Language Models...",single_hop_specifc_query_synthesizer
1,Wht is the role of English in LLMs?,[Based Development As a computer scientist and...,The grammar rules of programming languages lik...,single_hop_specifc_query_synthesizer
2,Wht r LLMs?,[Simon Willison’s Weblog Subscribe Stuff we fi...,"LLMs, or Large Language Models, are the latest...",single_hop_specifc_query_synthesizer
3,What ChatGPT do?,[easy to follow. The rest of the document incl...,ChatGPT was used to extract data into a table ...,single_hop_specifc_query_synthesizer
4,How do the ethics of AI training data impact t...,[<1-hop>\n\nCode may be the best application T...,The ethics of AI training data significantly i...,multi_hop_abstract_query_synthesizer
5,How does the black box nature of Large Languag...,[<1-hop>\n\nCode may be the best application T...,The black box nature of Large Language Models ...,multi_hop_abstract_query_synthesizer
6,How have the training costs and environmental ...,[<1-hop>\n\nCode may be the best application T...,"The training costs for AI models, particularly...",multi_hop_abstract_query_synthesizer
7,How OpenAI and AI ethics and legality related ...,[<1-hop>\n\nCode may be the best application T...,OpenAI was the first organization to release a...,multi_hop_abstract_query_synthesizer
8,How has Meta's approach to training data and m...,[<1-hop>\n\nreasoning patterns. Another common...,Meta's approach to training data and model acc...,multi_hop_specific_query_synthesizer
9,How has the development and criticism of ChatG...,[<1-hop>\n\nreasoning patterns. Another common...,The development of ChatGPT and other large lan...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [15]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [16]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [17]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [19]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [20]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [21]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [38]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

Each context item will be separated by "<|>" separator.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [39]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [42]:
from operator import itemgetter
import pdb
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.documents.base import Document
from langchain.schema import StrOutputParser

def parse_docs(docs: list[Document]) -> str:
    return "\n<|>\n".join([doc.page_content for doc in docs])

rag_chain = (
    {"context": itemgetter("question") | retriever | parse_docs, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [43]:
rag_chain.invoke({"question" : "What are Agents?"})

'The term "agents" refers to AI systems that can act on your behalf, but it lacks a single, clear, and widely understood meaning. There are different interpretations, such as AI agents being like travel agents, or being based on large language models (LLMs) that access tools to solve problems. However, there is confusion and skepticism about their practical utility, particularly due to challenges like "gullibility" in distinguishing truth from fiction. Overall, the concept of "agents" is seen as perpetually in development, with few examples in production despite excitement around the idea.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [46]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [47]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: this one is the simple comparing of what we expect and what we've got, so we can name it "correctness" check
- `labeled_helpfulness_evaluator`: here we have strict desc of what should be tested (`config.criteria.helpfulness`) and this task will be handled by testing LLM
- `dope_or_nope_evaluator`: the same is here, we'll try to evaluate how cool, modern, youthful our responses are

## LangSmith Evaluation

In [48]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'new-toy-45' at:
https://smith.langchain.com/o/de22f916-4054-4062-b948-9485f791faec/datasets/7f2342f1-59ce-42f8-bcb9-95ca0f94cf72/compare?selectedSessions=1d00002b-582e-41dd-bbf6-fd8cdfe3c738




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,In the context of 2023 being a breakthrough ye...,The ethical and legal challenges surrounding A...,None,"In 2023, Large Language Models (LLMs) marked a...",1,0,0,3.000795,b992d47e-308c-4fdd-b412-6ff6356489a1,b2fdf2ef-9e1b-43f6-9d67-fc796ecc296c
1,How has the development of Claude 3 and its su...,I don't know.,None,The development of Claude 3 and its subsequent...,0,0,0,1.329209,23677534-e916-40a4-834d-1563d62d1d35,129119ae-3784-4ea2-a695-2ffcf0375d70
2,How has the development and criticism of ChatG...,The development and criticism of ChatGPT and o...,None,The development of ChatGPT and other large lan...,1,1,0,5.515565,a1e4e5e1-6d79-4ef7-8011-2d29e58b3863,5e3b0880-3e49-4123-9b25-926fb77d9d89
3,How has Meta's approach to training data and m...,Meta's approach to training data and model acc...,None,Meta's approach to training data and model acc...,1,1,1,3.883086,7c22afc1-b72b-4e57-a8b2-f2cde8281b5f,418e1323-f287-4331-8e90-0c73142d40ce
4,How OpenAI and AI ethics and legality related ...,The legal arguments surrounding AI and LLMs ar...,None,OpenAI was the first organization to release a...,1,1,0,2.992061,101289fb-9ff0-4023-8f17-defc6120c5a3,176f8b0c-021a-4ac0-a774-d46f12a163f7
5,How have the training costs and environmental ...,Training costs for AI models have decreased si...,None,"The training costs for AI models, particularly...",1,0,0,4.073393,dd600d8f-2d5f-4da5-b236-1f1cfc05fdb1,9a5255f0-ad82-44f3-990d-edcb78f03478
6,How does the black box nature of Large Languag...,I don't know.,None,The black box nature of Large Language Models ...,0,0,0,1.118006,7a24ed2a-38cc-41e8-a314-432163f11436,64ae5cc0-7a1a-4b30-a2ed-1627671eb072
7,How do the ethics of AI training data impact t...,I don't know.,None,The ethics of AI training data significantly i...,0,0,0,0.949820,cd3e94d3-c839-4f8b-a487-078a6e58d681,67c637c9-d509-44a0-856f-5fd607549159
8,What ChatGPT do?,I don't know.,None,ChatGPT was used to extract data into a table ...,0,0,0,1.234366,8747fd45-bd2c-464f-b18b-67f0da7659e8,15dd43b7-fd28-494c-995d-be0ad1c501b5
9,Wht r LLMs?,I don't know.,None,"LLMs, or Large Language Models, are the latest...",0,0,0,1.798200,73ea8ee7-ca93-494a-aa51-5a54cb530842,6e1e23bc-a4c3-411b-936b-c2c37e4ec465


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [49]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

Each context item will be separated by "<|>" separator.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [50]:
rag_documents = docs

In [51]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [52]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [53]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [54]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [55]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever | parse_docs, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [56]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

"Agents, man, they’re this super vague term floating around in the AI world. Some folks see them as digital helpers that act on your behalf, like a travel agent booking your trips. Others think of them as LLMs (that’s language models) using tools in a loop to solve problems. But honestly, there’s no solid definition everyone agrees on. It feels like they’re always “coming soon,” but I’m skeptical about their real utility because these systems can be pretty gullible. Like, they can’t always tell truth from fiction, and that's a big deal if you want them making decisions for you. Until we crack that gullibility nut, these agent dreams might just be that—dreams!"

Finally, we can evaluate the new chain on the same test set!

In [57]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'definite-quiet-46' at:
https://smith.langchain.com/o/de22f916-4054-4062-b948-9485f791faec/datasets/7f2342f1-59ce-42f8-bcb9-95ca0f94cf72/compare?selectedSessions=2f238ea2-03d7-49ea-93c1-809ae815dfd0




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,In the context of 2023 being a breakthrough ye...,"Yo, 2023 was a wild ride for Large Language Mo...",None,"In 2023, Large Language Models (LLMs) marked a...",1,1,1,9.499848,b992d47e-308c-4fdd-b412-6ff6356489a1,7aa7cade-f7de-4474-81bf-cb146f4ca080
1,How has the development of Claude 3 and its su...,"Yo, the Claude 3 series from Anthropic has ser...",None,The development of Claude 3 and its subsequent...,1,0,1,5.014262,23677534-e916-40a4-834d-1563d62d1d35,e6663691-d04e-4220-86b4-78f000b10064
2,How has the development and criticism of ChatG...,"Yo, so here's the lowdown on the evolution of ...",None,The development of ChatGPT and other large lan...,1,0,1,8.147474,a1e4e5e1-6d79-4ef7-8011-2d29e58b3863,aedba667-fcd6-4627-859c-261419a7293c
3,How has Meta's approach to training data and m...,Meta's approach to training data has seriously...,None,Meta's approach to training data and model acc...,1,0,1,3.135684,7c22afc1-b72b-4e57-a8b2-f2cde8281b5f,e193bc8b-06ef-43d7-a66c-dcc23f21c188
4,How OpenAI and AI ethics and legality related ...,"Yo, in the wild world of LLMs, OpenAI’s cookin...",None,OpenAI was the first organization to release a...,1,0,1,7.201817,101289fb-9ff0-4023-8f17-defc6120c5a3,a42cc83f-5e0d-427e-bd11-09491afdf80e
5,How have the training costs and environmental ...,"Yo, let’s break this down! Training costs for ...",None,"The training costs for AI models, particularly...",1,0,1,4.372936,dd600d8f-2d5f-4da5-b236-1f1cfc05fdb1,cfaa07e6-6be8-4433-b049-5a2c412fc8ca
6,How does the black box nature of Large Languag...,The black box vibe of LLMs makes their develop...,None,The black box nature of Large Language Models ...,1,1,1,4.242840,7a24ed2a-38cc-41e8-a314-432163f11436,6417c4a0-5bbf-45ef-8f7b-31dcb4b52ff5
7,How do the ethics of AI training data impact t...,"Yo, the ethics of AI training data are like a ...",None,The ethics of AI training data significantly i...,1,1,1,4.806457,cd3e94d3-c839-4f8b-a487-078a6e58d681,ba28cc3c-812e-41d0-910f-472a4036eaad
8,What ChatGPT do?,"ChatGPT is like a super slick word calculator,...",None,ChatGPT was used to extract data into a table ...,0,0,1,2.907211,8747fd45-bd2c-464f-b18b-67f0da7659e8,0d728250-12a9-4a0f-96d8-2549da6a07c6
9,Wht r LLMs?,"LLMs, or Large Language Models, are basically ...",None,"LLMs, or Large Language Models, are the latest...",1,1,1,7.338888,73ea8ee7-ca93-494a-aa51-5a54cb530842,16bfe892-6e70-4173-a916-001e690f1ec7


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.